In [3]:
from optseq import Model, Mode, Parameters, Activity, Resource

In [44]:
duration = {"A1": 30, "A2": 30, "B1": 20, 'B2': 20}

setup = {
    ("A1", "A2"): 10, ("A1", "B1"): 0, ("A1", "B2"): 0, ("A1", "start"): 0,
    ("A2", "A1"): 20, ("A2", "B1"): 0, ("A2", "B2"): 0, ("A2", "start"): 0,
    ("B1", "A1"): 0, ("B1", "A2"): 0, ("B1", "B2"): 30, ("B1", "start"): 0,
    ("B2", "A1"): 0, ("B2", "A2"): 0, ("B2", "B1"): 40, ("B2", "start"): 0,
    ("start", "A1"): 0, ("start", "A2"): 0, ("start", "B1"): 40, ("start", "B2"): 0,
}

state_dict = {"start": 0, "A1": 1, "A2": 2, "B1": 3, 'B2': 4}

In [45]:
model = Model()

In [46]:
rs1 = model.addResource("line1", 1)
rs2 = model.addResource("line2", 1)

In [47]:
act = {}
mode = {}
for demand in duration:
    act[demand] = model.addActivity(f"Act[{demand}]")
    mode[demand] = Mode(f"Mode[{demand}]", duration[demand])
    if demand[0] == 'A':
        mode[demand].addResource(rs1, {(0, "inf"): 1})
        act[demand].addModes(mode[demand])
    elif demand[0] == 'B':
        mode[demand].addResource(rs2, {(0, "inf"): 1})
        act[demand].addModes(mode[demand])

In [48]:
state = model.addState("Setup_State")
state.addValue(time=0, value=state_dict["start"])

In [49]:
# setup mode
mode_setup = {}
for (i, j) in setup:
    mode_setup[i, j] = Mode(f"Mode_setup_{i}_{j}", setup[i, j])
    mode_setup[i, j].addState(state, state_dict[i], state_dict[j])
    if setup[i, j] != 0:
        if j[0] == 'A':
            mode_setup[i, j].addResource(rs1, {(0, setup[i, j]): 1})
        elif j[0] == 'B':
            mode_setup[i, j].addResource(rs2, {(0, setup[i, j]): 1})
        else:
            mode_setup[i, j].addResource(rs1, {(0, setup[i, j]): 1})
            mode_setup[i, j].addResource(rs2, {(0, setup[i, j]): 1})

In [50]:
act_setup = {}
for k in duration:
    act_setup[k] = model.addActivity(f"Setup[{k}]", autoselect=True)
    for (i, j) in setup:
        if k == j:
            act_setup[k].addModes(mode_setup[i, j])

In [51]:
# temporal (precedense) constraints
for j in act_setup:
    model.addTemporal(act_setup[j], act[j], "CS")
    model.addTemporal(act[j], act_setup[j], "SC")

In [52]:
model.Params.TimeLimit = 10
model.Params.Makespan = True

In [53]:
model.optimize()


 ================ Now solving the problem ================ 


Solutions:
    source   ---     0     0
      sink   ---    60    60
   Act[A1]   ---     0    30
   Act[A2]   ---    30    60
   Act[B1]   ---     1    21
   Act[B2]   ---    31    51
 Setup[A1] Mode_setup_start_A1     0     0
 Setup[A2] Mode_setup_B1_A2    30    30
 Setup[B1] Mode_setup_A1_B1     1     1
 Setup[B2] Mode_setup_A2_B2    31    31
